In [1]:
# Setup iniziale con i vari import da sfruttare per i due svolgimenti separati
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [2]:
# Caricamento Dataset di Kaggle
# Kaggle "DOM_hourly": columns => ["Datetime", "DOM_MW"]
df = pd.read_csv("DOM_hourly.csv", parse_dates=["Datetime"])
# Forziamo che la colonna 'Datetime' sia in formato datetime
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.set_index('Datetime', inplace=True)

In [3]:
# Feature engineering
df["hour"] = df.index.hour
df["dayofweek"] = df.index.dayofweek
df["month"] = df.index.month

In [5]:
# Calcolo delle varie medie, settimanale, mensile e globale
media_settimanale = df['DOM_MW'].resample('W').mean()
media_mensile = df['DOM_MW'].resample('ME').mean()
media_globale = df['DOM_MW'].mean()

# Aggiungere le medie al DataFrame
df['Media_Settimanale'] = media_settimanale.reindex(df.index).ffill()
df['Media_Mensile'] = media_mensile.reindex(df.index).ffill()
df['Media_Globale'] = media_globale

### SVOLGIMENTO RAPIDO USANDO DIRETTAMENTE PANDAS

### SVOLGIMENTO CON MODELLO DI MACHINE LEARNING

In [ ]:
# Target binario: 1 se consumo sopra la mediana
df["target"] = (df["AEP_MW"] > df["AEP_MW"].median()).astype(int)

# #### Selezione delle feature
X = df[["hour", "dayofweek", "month"]]
y = df["target"]

# #### Split del dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.25, random_state=42
)

# #### Modello 1 – Decision Tree
tree = DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

print("Decision Tree:")
print(classification_report(y_test, y_pred_tree, digits=3))

# #### Modello 2 – Neural Network (MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred_mlp = mlp.predict(X_test_scaled)

print("Neural Network:")
print(classification_report(y_test, y_pred_mlp, digits=3))